# 충전소 입지 선정 불가 후보지 제거

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm 

In [48]:
grid_df=gpd.read_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인 인구정보.geojson')
# 설치 불가지역 QGIS로 geometry 취합 처리
drop_df=gpd.read_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/설치불가지역(수질보호공공수역제외).geojson')
drop_df.to_crs('EPSG:5178',inplace=True)
grid_df.to_crs('EPSG:5178',inplace=True)

In [3]:
print(len(grid_df),len(drop_df))

60300 7666


In [5]:
grid_df.head()

,gid,lbl,val,longitude,latitude,geometry
0,다사748072,N/A,0.0,127.216732,37.163656,"MULTIPOLYGON (((127.21617 37.16320, 127.21617 ..."
1,다사751059,N/A,0.0,127.220154,37.151945,"MULTIPOLYGON (((127.21959 37.15149, 127.21959 ..."
2,다사750101,N/A,0.0,127.218888,37.189802,"MULTIPOLYGON (((127.21833 37.18935, 127.21832 ..."
3,다사914045,N/A,0.0,127.403730,37.139617,"MULTIPOLYGON (((127.40317 37.13917, 127.40317 ..."
4,다사918030,N/A,0.0,127.408251,37.126099,"MULTIPOLYGON (((127.40769 37.12565, 127.40769 ..."


In [49]:
# 격자 geometry centroid로 처리
tqdm.pandas()
grid_df['geometry']= grid_df.progress_apply(lambda row : row['geometry'].centroid,axis=1)

100%|██████████| 60300/60300 [00:01<00:00, 31629.13it/s]


In [19]:
drop_df.head()

,MNUM,ALIAS,REMARK,NTFDATE,Shape_Leng,Shape_Area,CODE_NM,layer,path,geometry
0,15000004146100000000UQF1320024000,None,None,None,470.744027,3314.170394,시가지경관지구(일반),연속주제_국토계획_경관지구,/Users/hansolyu/Downloads/41461/연속주제_국...,"MULTIPOLYGON (((974445.078 1914326.210, 974431..."
1,15000004146100000000UQF1200010000,None,None,None,1119.067692,78193.713141,특화경관지구,연속주제_국토계획_경관지구,/Users/hansolyu/Downloads/41461/연속주제_국...,"MULTIPOLYGON (((971072.978 1926458.971, 971065..."
2,40500004146020210623UQF1310005001,None,None,None,1170.893880,7406.130254,시가지경관지구(중심),연속주제_국토계획_경관지구,/Users/hansolyu/Downloads/41461/연속주제_국...,"MULTIPOLYGON (((974394.360 1914791.915, 974409..."
3,40500004146020150497UQF1320034001,None,None,None,386.359245,2004.161315,시가지경관지구(일반),연속주제_국토계획_경관지구,/Users/hansolyu/Downloads/41461/연속주제_국...,"MULTIPOLYGON (((972546.133 1915594.708, 972543..."
4,15000004146100000000UQF1200012000,None,None,None,1323.370577,92362.358917,특화경관지구,연속주제_국토계획_경관지구,/Users/hansolyu/Downloads/41461/연속주제_국...,"MULTIPOLYGON (((971729.344 1926176.151, 971724..."


In [51]:
tqdm.pandas()
# Drop 할 polygon 안에 들어가는 points 에 drop column 추가
result = gpd.sjoin(grid_df,drop_df,how='left',op='within')
# Drop table 과 join 안된 points 만 남기기
result=result[result['CODE_NM'].isna()]
result=result.drop(columns=drop_df.drop(columns=['geometry']).columns.values)
result=result.drop(columns=['index_right'])
result.fillna(0,inplace=True)
# result.rename(columns={'geometry':'grid'},inplace=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [52]:
result

,gid,lbl,val,longitude,latitude,geometry
2,다사750101,N/A,0.0,127.218888,37.189802,POINT (975240.056 1909843.864)
3,다사914045,N/A,0.0,127.403730,37.139617,POINT (991640.264 1904244.063)
4,다사918030,N/A,0.0,127.408251,37.126099,POINT (992040.289 1902744.057)
5,다사595284,N/A,0.0,127.043257,37.354210,POINT (959739.665 1928143.776)
6,다사720059,N/A,0.0,127.185244,37.151858,POINT (972240.096 1905643.788)
...,...,...,...,...,...,...
60281,다사750282,0,0.0,127.218280,37.352953,POINT (975239.789 1927944.005)
60284,다사631163,0,0.0,127.084506,37.245294,POINT (963339.873 1916043.736)
60294,다사581288,0,0.0,127.027427,37.357753,POINT (958339.648 1928543.759)
60297,다사588268,0,0.0,127.035442,37.339757,POINT (959039.684 1926543.753)


In [53]:
result.to_file('/Users/jacob/Desktop/VS_Code_Repository/DeepLearning_Alpaco/빅콘테스트2022/Raw data/용인시_Grid_filtered(xyfeatures).geojson',encoding='utf-8',index=False)